## Helper methods for demonstration

In [1]:
#nullable enable

using System.Diagnostics;
using System.IO;
using System.Runtime.InteropServices;

Process CreateInstance(string? username = null)
{
    //Arguments that are specific to the Java (ChatApp) application.
    List<string> appArguments = new();
    if (username is not null)
    {
        appArguments.AddRange(new string[]
        {
            "127.0.0.1",
            (8080).ToString(),
            username
        });
    }

    //Arguments for the Java runtime process.
    List<string> processArguments = new()
    {
        "-jar",
        "\"" + Path.Combine(
            Environment.CurrentDirectory,
            Path.GetFileName(Environment.CurrentDirectory) + ".jar") + "\"",
    };
    processArguments.AddRange(appArguments);

    ProcessStartInfo processStartInfo = new();
    processStartInfo.FileName = "java.exe";
    processStartInfo.Arguments = string.Join(" ", processArguments);
    processStartInfo.WorkingDirectory = Environment.CurrentDirectory;
    processStartInfo.UseShellExecute = true; //Cause the console to appear.

    Process process = new Process();
    process.StartInfo = processStartInfo;
    process.Start();

    return process;
}

## A: Scenario Demonstration

### 1. Creating a server and three clients

Firstly I will create three peers as requested by the scenario. One of these peers will become the coordinator/host, the other two will become clients.

For this first example I will use command line arguments to set the options for each peer, I will then later on show creating an instance of the app using the UI.  
The command line usage for setting the options is as follows: `<IP> <PORT> <DESIRED_USERNAME>`.

In [70]:
CreateInstance("Jack");
// CreateInstance("Bob");
CreateInstance("Bob");
CreateInstance("Alice");

### 2. Demonstrate how the coordinator works

Next I will briefly demonstrate how the coordinator works, the coordinator, or host as I like to call it, is the peer that is responsible for managing all other peers and relaying messages between them, amongst a few other network related events.

The first thing that happens when a client connects is they send a handshake to the host, this handshake holds the desired username that the new peer would like to use, if the username is already taken then the host will add a suffix to the username, additionally the host will assign the new peer a unique ID. This information is then sent back to the new peer at which point all of the other peers are notified of the new peer as well as the new peer requesting the information of all of the other peers.  
See [ChatManager.java:419](./src/chat_app/backend/ChatManager.java#L419) for the code that handles the handshake.

The host is also responsible for relaying messages between clients, for example when a client wants to message another client, the host will receive a `NetMessage` which contains a `Message` type payload. This payload contains the ID of the client to be messaged and the message itself. The host will then add in an extra field which contains the ID of the client that sent the message, we do this from the host so that the sender can't impersonate another client. The host will then relay all of this information to the client that was messaged.  
See [ChatManager.java:618](./src/chat_app/backend/ChatManager.java#L618) for the code that handles relaying messages.

Another thing that the host does is send out ping messages to all clients at a set interval, this is used to check if a client has timed-out. If a client doesn't respond with a pong message within a set time then the host will assume that the client has timed-out and will remove the client from the list of peers as well as indicate to the other peers that the client has left.  
See [PingPong.java:35](./src/chat_app/backend/PingPong.java#L35) for the code that handles the host ping/pong events.

### 3. Sending a message from one of the clients and replying back using private and broadcast messaging

I will now demonstrate a working example of sending messages between peers using broadcast and private messaging.

Starting with broadcast messages, we can see that on my UI we have a tab on the left named "Global", this tab is where all of the broadcast messages are displayed. If I send a message in here we can see that all other peers have received the message.  

Next, if you wanted to send a private message, you can click on the name of the peer you want to message in the list on the left, this will then change the contents of the chat box to display your message history with that specific peer. If I now write a message and send it we can see that the target peer has gotten a notification next to the sender name indicating that they have an unread message from them. If we click onto the target peer's tab we can see that the message has been received, additionally if we check the other peers we can see that they never received the private message.

### 4. Disconnecting a peer that is not the coordinator

Next I will show what happens when a client peer is disconnected.

If I close one of these client windows, we can see that the other two peers have removed the disconnected peer from their list of peers. Additionally, a system message will be logged into each of the peers global chat tab indicating that the peer has disconnected.  
Behind the scenes when a client is disconnected, the server will send out a `NetMessage` with a payload type of `Peer` to all other clients. This message will contain a status update about the disconnected peer, in this case the status will be set to `DISCONNECTED`. The other clients will then receive this message and update their list of peers accordingly.  
See [ChatManager.java:703](./src/chat_app/backend/ChatManager.java#L703) for the server code that handles the peer status update.

### 5. Introducing a new client

I will now introduce a new client to the network. This time however I will not pass any command line arguments to the app, we can then see how the client would be configured via the startup UI.

In [57]:
// CreateInstance("Sam");
CreateInstance(); //Configure the instance manually via the UI.

Similarly to the way we provided the initial server IP address, port number and desired username on the command line, we can provide these values via the UI.

When a new peer wants to connect to the network, we will search for an existing host at the specified address and port. If the host is found then we will follow the same procedure as before where the client sends a handshake to the server to set it's username and receive a unique ID as well as information about the other peers. And like with the disconnect event we can see that a system message has appeared in the global chat tab of each peer indicating that the new peer has connected.  
See [ChatManager.java:340](./src/chat_app/backend/ChatManager.java#L340) for the client code that handles the connection initialization and also see [ChatManager.java:479](./src/chat_app/backend/ChatManager.java#L479) for the client code that handles the handshake response.

### 6. Disconnecting the coordinator

For the last part of this section I will show what happens when the coordinator/host is disconnected.

If I close the coordinator window the first thing that we will see happen is each of the clients chat boxes become disabled as well as their peer lists being cleared. A few moments later the chat boxes on each peer will be re-enabled and the peer lists will be populated with the remaining peers.  
Behind the scenes what is happening here is that when the server disconnects each of the clients will trigger a restart of the connection process, however we don't clear the peers list yet as we will use this information later during the reconnection process.  
The first thing we do during the reconnection is wait for a time between 0 and 1.5 seconds, this time is determined by our own ID. The reason for doing this is to prevent race conditions when multiple clients are trying to reconnect at the same time, while this is not a foolproof method, it greatly reduces the chance of reconnection issues. If I went back and modified the program I would have the coordinator tell all peers who the next host would be before disconnecting, I would then keep this current method for when the coordinator disconnects unexpectedly.  
After we have waited for the random amount of time, we will use our old peers list to search for a new host, we use the IP's of each old peer to see if they have a joinable server. If they do then we will connect to them. If we fail to connect to any of the old peers then we fall back to checking the original server IP address that was specified when we first started the application. If we still fail to connect to a server instance then we will start hosting our own server instance and in-turn becoming the new coordinator.  
See [ChatManager.java:155](./src/chat_app/backend/ChatManager.java#L155) for the code that handles the reconnection process.

## B: Implementation Inspection

### 1. Design patterns

I will now show some example of the design patterns that I have used in this project.

The core code that handles networking in this project consists of four files, two of these handle server side logic, one handles client side logic and the last is a virtual/abstract class that both the server and client extend. My reason for taking this abstract approach was to reduce the amount of duplicated code, this is because the way host and client sockets work in java are very similar, and so I can write the same base code for both classes and then extend or override methods as needed.  
See [ASocket.java](./src/readiefur/sockets/ASocket.java) for the abstract class that both the server and client extend.

I had considered making the `ChatManager` class a singleton class, this would make sense as there would only realistically be one instance of this class per application, however I decided against doing this as it would make unit testing easier as well as allowing for multiple for multiple peers to run under the same process if this were to be needed in the future.

There are also a few times I make use of protected variation, an example of this can be seen in my [ServerPeer](./src/chat_app/backend/ServerPeer.java) class. This class is an extension of the `Peer` class, however it exposes some variables of the `Peer` class that should only be modified by a server instance.

Another pattern I have used is the factory patten. This is used twice in my code, once for the creation of the `ServerClientHost` class, which is responsible for handling a single connection to another peer, the other time I use it is for my UI parser.

I also make use of many decorator patterns, particularly in my UI builder. If we take a look at the file [ConfigurationUI.java](./src/chat_app/frontend/ConfigurationUI.java#L93) we can see that I have added many property decorators. These decorators are used to tell the UI builder how to build certain parts of the UI or how to handle certain events. For example if we take a look at line `93` we can see that the there is an attribute called `EventCallbackAttribute` which is attached to the method `Connect_Click`, the use of this decorator tells the UI builder that when the `Connect` button is clicked it should call the method `Connect_Click`.

### 2. Modularity/component design

Moving onto modularity and component design.

When I write programs I like to try and make them as modular as possible, meaning that no one part of the program is too tightly coupled to another. This can then help me with reusability of parts of my program as well as even being able to incorporate parts of my program into other projects.  
I have split this program into two main packages, the first is the `readiefur` package which contains all modular code that is not specific to this project, and so can be used in other projects without any issues. The second package is the `chat_app` package which contains all the code that is specific to this project.

If we first take a look at some of the files inside of the `readiefur` package, lets say the file [XMLUI.java](./src/readiefur/xml_ui/XMLUI.java) we can observe that from the imports used in this file, only build-in Java libraries and those of the `readiefur.xml_ui` package are used. This shows that this file is not dependent on any other files in the project, and so can be used in other projects.

One example of a component in this project, amongst many, is the [ServerClientHost.java](./src/readiefur/sockets/ServerClientHost.java) class. This class is responsible for handling a single connection to another peer. We would expect many many clients to connect to a single server and so we need a way to handle each individual connection. This component class has been built in such a way that it can be created multiple times, handling each connection to another peer. Of course we would need something to manage each of these instances which is where the `ServerManager` class would come in.

### 3. Unit testing

I will next show a few example of the unit tests that I have written for this project.

Originally I was going to write some unit tests for the frontend (UI) of this project, my intention was to make use of the Windows Accessibility API to automate the UI, however after a quick inspection I noticed that no elements were appearing in the UI tree, so I did a quick bit of research and found out that the Java Swing framework does not comply with the Windows Accessibility API as it uses its own rendering engine. As a result of this I have not written any automated unit tests for the frontend of this project.

I have however written a few unit tests that cover the backend of this project. If we take a look at the file [testing/Backend.java](./src/testing/Backend.java) we can see that I have written a few unit tests related around the `ChatManager` class.  

I will show you a simple test I have written which will create two instances of the `ChatManager` class, one will be the host and the other will be a client. The test will check that one instance became the host and the other became a client.  
See line `62` for this test.

A more complex test I have written is one that will check if a private message is received by the correct peer. This test will create three instances of the `ChatManager` class, one will be the host and the other two will be clients. The test will then send a private message from a peer to another peer and it will check that only the target peer received the message.  
See line `196` for this test.

## C: Additional points

### 1. Additional features and subjects of interest

Moving on to some features that I have added in addition to this projects requirements.

I have added a few extra features such as the Log manager, which is only really intended for extra debugging purposes. However the main additional feature I want to focus on is my `readiefur.xml_ui` package.  
This package contains code for building Java Swing UI's from an XML file. The reason behind me wanting to add such a feature was because I wasn't a huge fan of the way Java Swing UI's are built through code as it seemed to be VERY repetitive. The other reason was that I am used to how UI's are built in C#'s XAML framework. So I decided to make a simplified version of this framework in Java, it was also an excuse for me to implement some extra features of the Java language to this project such as reflection and annotations.  
My simplified XML UI builder has support for Grid aligned layouts, property decorators, event callbacks and one-way-binding from the code to the UI.

The UI building starts within the [XMLUI](./src/readiefur/xml_ui/XMLUI.java) class. All custom elements will extend from this virtual class and specify the root component type as the generic parameter. Then using reflection, the virtual class will search for an xml file with the same class name as the one that extends it.

The next file that we should focus on is the [UIBuilderFactory](./src/readiefur/xml_ui/factory/UIBuilderFactory.java) class. This class is where the XML file will be parsed into a tree of Java Swing `Component` objects. Inside of here we have a recursive method called `ParseXMLNode` which is the first stage in converting an XML node into a Java Swing `Component` object. This method will apply common properties such as the `name` of the element, configure any binding attributes, as well as applying any custom properties that it finds on the corresponding Java class for that XML node.  
After that the method will call the `ParseChildTree` on the component wrapper which will then compute any other properties for that element and then recursively call the `ParseXMLNode` method on each child node if applicable.  
Once this has all completed the root Java Swing `Component` object will be returned and then set as the `rootComponent` variable of the `XMLUI` class which the extended class can call upon.